# Data Analysis and Visualisation &mdash; Lab 01

## Problem 1: How has Linux adoption as a desktop operating system changed up to 2025?

### 1. Objective

The objective of this study is to examine the trend of Linux adoption as a desktop operating system up to 2025. By collecting data from public sources (*i.e.* StatCounter, Steam Hardware Survey, DistroWatch), we aim to observe whether Linux usage among desktop users is increasing, or remaining stable over time.

### 2. Effective Questions

1. How has Linux desktop market share evolved globally and regionally (*i.e.* Viet Nam) over time compared to Windows and macOS?
2. Does Linux show faster adoption in gaming (*i.e.* Steam) than in general desktop usage?
3. Are changes in DistroWatch popularity rankings associated with measurable shifts in Linux usage share?

### 3. Variables of Interest

| Variable | Description | Type | Source |
| --- | --- | --- | --- |
| `date` | Month and year of observation | Datetime | StatCounter, Steam Hardware Survey |
| `region` | Geographic region or country | Categorical | StatCounter |
| `linux_market` | % market share of desktop users using Linux | Numeric | StatCounter |
| `win_market` | % market share of desktop users using Windows | Numeric | StatCounter |
| `mac_market` | % market share of desktop users using macOS | Numeric | StatCounter |
| `linux_steam` | % of Steam users on Linux | Numeric | Steam Hardware Survey |
| `win_steam` | % of Steam users on Windows | Numeric | Steam Hardware Survey |
| `mac_steam` | % of Steam users on macOS | Numeric | Steam Hardware Survey |
| `distro_month_range` | Month duration of observation | Numeric | DistroWatch |
| `distro_hpd` | Page hits per day for Linux distros (popularity indicator) | Numeric | DistroWatch |
| `distro_rank` | Rankings of Linux distros based on page hits per day | Numeric | DistroWatch |

### 4. Collecting Data

#### 4.1. StatCounter

In [1]:
import urllib.parse
import pandas as pd
import pycountry

def get_statcounter_os_share(start = "2023-01", end = "2025-10", country = "ww"):
    region = pycountry.countries.get(alpha_2=country).name if country != "ww" else "Worldwide"
    region_enc = urllib.parse.quote(region)
    
    csv_url = (
        f"https://gs.statcounter.com/os-market-share/desktop/chart.php"
        f"?device=Desktop"
        f"&device_hidden=desktop"
        f"&statType=Operating%20System"
        f"&statType_hidden=os_combined"
        f"&region={region_enc}"
        f"&region_hidden={country}"
        f"&granularity=monthly"
        f"&fromInt={start.replace('-', '')}"
        f"&fromMonthYear={start}"
        f"&toInt={end.replace('-', '')}"
        f"&toMonthYear={end}"
        f"&csv=1"
    )

    df = pd.read_csv(csv_url)
    df["Region"] = region
    return df

statcounter_ww = get_statcounter_os_share()
statcounter_vn = get_statcounter_os_share(country="VN")

statcounter_ww.to_csv("../data/StatCounter_Worldwide.csv", index=False)
statcounter_vn.to_csv("../data/StatCounter_Viet+Nam.csv", index=False)

#### 4.2. Steam Hardware Survey

In [4]:
steam_hw_survey = pd.read_csv("https://raw.githubusercontent.com/jdegene/steamHWsurvey/refs/heads/master/shs_platform.csv")
steam_hw_survey = steam_hw_survey[(steam_hw_survey["date"] >= "2023-01-01") & (steam_hw_survey["date"] <= "2025-10-01")]

platform_versions = steam_hw_survey[steam_hw_survey["category"].isin(["Windows Version", "OSX Version", "Linux Version"])]

platform_versions.to_csv("../data/Steam+HW+Survey_Platform+Versions.csv", index=False)

#### 4.3. DistroWatch

In [5]:
def get_distrowatch_phr():
    url = "https://distrowatch.com/dwres.php?resource=popularity"
    df = pd.DataFrame()
    
    for table in pd.read_html(url):
        cols = table.columns.astype(str)
        
        if any("Last" in col for col in cols):
            month_range = cols[0].split(" ")[1]
            
            table.columns = ["ranking", "distro", "hits_per_day"]
            table["month_range"] = month_range

            df = pd.concat([df, table], ignore_index=True)
            
    return df

distrowatch_phr = get_distrowatch_phr()
distrowatch_phr.to_csv("../data/DistroWatch_HPD.csv", index=False)

## Problem 2: Which factors drive customer preference toward regional brands in Viet Nam's coffee market?

### 1. Objective

The objective of this study is to study how the growing availability and convenience of e-books affect the usage and sales of physical books by understanding whether digital reading trends are reducing physical book purchases or merely expanding overall reading habits.

### 4. Collecting Data

#### 4.1. Google Maps (Playwright)

In [ ]:
%run ../utils/google-maps.py

#### 4.2. Coffee chain prices

In [ ]:
def to_float(s):
    try:
        return float(s.replace(".", "").replace(" VNĐ", "").replace(",", ""))
    except:
        return None


def split_name_and_size(raw):
    clean = " ".join(raw.split())
    match = re.search(r"\((S|M|L)\)$", clean)
    if match:
        size = match.group(1)
        name = clean[:match.start()].strip()
    else:
        size = None
        name = clean
    return name, size


def get_starbucks_aeon_mall_binh_tan_prices():
    url = "https://aeonmall-binhtan.com.vn/thuc-don/thuc-don-starbucks"
    res = requests.get(url).text
    soup = Soup(res, "html.parser")

    records = []
    sizes_map = {
        "Short": "S",
        "Tall": "M",
        "Grande": "L",
        "Venti": "XL",
    }

    for table in soup.find_all("table"):
        header_cols = [th.text.strip() for th in table.find_all("tr")[0].find_all("td")]
        sizes = header_cols[1:]

        for row in table.find_all("tr")[1:]:
            columns = row.find_all("td")
            item_name = columns[0].text.strip()

            for i, size in enumerate(sizes, start=1):
                size = size.split()[0]

                price_str = columns[i].text.strip()
                price = to_float(price_str) if price_str else None

                records.append({
                    "brand": "Starbucks",
                    "name": item_name,
                    "size": sizes_map[size],
                    "price": price,
                })

    df = pd.DataFrame(records)
    return df

In [ ]:
def get_highlands_prices():
    url = "https://vngia.vn/menu-highland-coffee/"
    res = requests.get(url).text
    soup = Soup(res, "html.parser")

    records = []
    table = soup.find("table", id="tablepress-8")
    for row in table.find_all("tr"):
        columns = row.find_all("td")

        if len(columns) != 2:
            continue

        raw_name = columns[0].text.strip()
        item_name, size = split_name_and_size(raw_name)
        price = to_float(columns[1].text.strip())

        records.append({
            "brand": "Highlands",
            "name": item_name,
            "size": size,
            "price": price,
        })

    df = pd.DataFrame(records)
    return df

In [ ]:
def get_phuc_long_prices():
    slugs = [
        "bst-tokubetsu-matcha--c011101",
        "tra-trai-cay--c01190",
        "tra-sua--c01187",
        "kem-silky--c01191",
        "ca-phe--c01189",
        "da-xay--c01193",
        "tra-nguyen-ban--c011102",
        "banh-lanh--c01194",
        "banh-cookies-croissant--c01195",
        "banh-mi--c01196"
    ]
    url = "https://api-crownx.winmart.vn/plg/api/web/item/category"

    records = []
    for slug in slugs:
        params = {"pageNumber": 1, "pageSize": 40, "slug": slug}
        res = requests.get(url, params).json()

        for item in res["data"]["items"]:
            base_price = item["salePrice"]

            for product in item["sizes"]:
                item_name = " ".join(product["name"].split()[:-1])
                price = base_price + product["addSalePrice"]

                records.append({
                    "brand": "Phúc Long",
                    "name": item_name,
                    "size": product["size"],
                    "price": price,
                })

    df = pd.DataFrame(records)
    return df

## Problem 3: Which laptop under 20 million VND offers the best value for Computer Science workloads such as model training or data analysis?